In [1]:
from haystack_integrations.document_stores.pgvector import PgvectorDocumentStore

document_store = PgvectorDocumentStore(
    table_name="haystack_docs",
    embedding_dimension=768,
    vector_function="cosine_similarity",
    recreate_table=True,
    search_strategy="hnsw",
)

In [5]:
from haystack import Pipeline
from haystack.components.converters import TextFileToDocument
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.components.embedders.ollama import OllamaDocumentEmbedder
from haystack_integrations.components.embedders.ollama import OllamaTextEmbedder

from dotenv import load_dotenv
load_dotenv()


file_paths = ["./data/agents.md", "./data/logits.md"]

indexing = Pipeline()
indexing.add_component("converter", TextFileToDocument())
indexing.add_component("embedder", OllamaDocumentEmbedder())
indexing.add_component("writer", DocumentWriter(document_store, DuplicatePolicy.OVERWRITE))
indexing.connect("converter", "embedder")
indexing.connect("embedder", "writer")
indexing.run({"converter": {"sources": file_paths}})


embedder = OllamaTextEmbedder()
result = embedder.run(text="What do llamas say once you have thanked them? No probllama!")
print(result['embedding'])


Calculating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.36s/it]


[1.4799904823303223, 0.5603218078613281, -2.8820483684539795, -1.3634796142578125, 0.6062513589859009, 0.2637723684310913, -0.6462752819061279, 0.028760356828570366, -0.37733298540115356, 0.6073045134544373, -0.4523448646068573, 1.0284909009933472, 0.8306690454483032, 0.9659475684165955, 1.004835605621338, -0.1461051106452942, 0.9330642819404602, -1.081034541130066, -0.4587674140930176, 0.49964454770088196, -1.679674506187439, -0.8013893365859985, -0.8434707522392273, -0.6815372109413147, 1.6872475147247314, -0.3760850727558136, -0.9182426333427429, 1.83568274974823, -0.29383584856987, -0.7610524892807007, 0.6017246246337891, -0.4551717936992645, -0.2224707305431366, -0.588925302028656, 0.40966519713401794, 0.6980066895484924, 1.258554458618164, -0.24508064985275269, 0.6248601675033569, -0.274873822927475, 1.1718494892120361, 0.38860562443733215, -0.3627476096153259, -0.005816969089210033, 1.0170842409133911, -0.3257341682910919, 0.556994616985321, 0.918053388595581, 1.3366992473602295

In [6]:
from haystack.components.builders import PromptBuilder

# All variables optional (default to empty string)
builder = PromptBuilder(
    template="Hello {{name}}! {{greeting}}",
    required_variables=[]  # or omit this parameter entirely
)

# Some variables required
builder = PromptBuilder(
    template="Hello {{name}}! {{greeting}}",
    required_variables=["name"]  # 'greeting' remains optional
)


In [11]:
import dspy
from dspy import Signature, InputField, OutputField, Module


lm = dspy.LM('ollama_chat/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

# Step 1: Define DSPy Signature
class AnalyzeMargins(Signature):
    context = InputField(desc="Relevant financial data")
    question = InputField(desc="User's trading question")
    answer = OutputField(desc="Insightful, accurate answer")

# Step 2: Create a Module using the Signature
class MarginAnalyzer(Module):
    def __init__(self):
        super().__init__()
        self.chain = dspy.Predict(AnalyzeMargins)

    def forward(self, context, question):
        return self.chain(context=context, question=question)



In [ ]:
import litellm
litellm._turn_on_debug()
import logging
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', 
                    handlers=[logging.FileHandler('vector_db.log', 'w', 'utf-8')])

context = '''
Tesla Inc Income Statement:
Q1 Revenue: $23B, Operating Income: $2.3B
Q2 Revenue: $24B, Operating Income: $2.5B
Q3 Revenue: $25B, Operating Income: $2.4B
'''
question = "Is Tesla's operating margin improving?"

analyzer = MarginAnalyzer()
result = analyzer(context=context, question=question)
print("📊 Answer:", result.answer)


📊 Answer: To determine if Tesla's operating margin is improving, we need to calculate the operating margin for each quarter provided and analyze the trend. Operating margin is calculated as Operating Income divided by Revenue, expressed as a percentage.

**Q1 Calculation:**
Operating Margin = (Operating Income / Revenue) * 100
= ($2.3B / $23B) * 100 ≈ 10.04%

**Q2 Calculation:**
Operating Margin = ($2.5B / $24B) * 100 ≈ 10.42%

**Q3 Calculation:**
Operating Margin = ($2.4B / $25B) * 100 ≈ 9.60%

**Analysis of Trend:**
- Q1: ~10.04%
- Q2: ~10.42% (increase from Q1)
- Q3: ~9.60% (decrease from Q2)

The operating margin increased slightly from Q1 to Q2 but then decreased in Q3. However, the overall trend shows a slight improvement over the three quarters despite the dip in Q3. This suggests that Tesla's operating margin is improving on average, with some volatility.

**Conclusion:**
Yes, Tesla's operating margin is improving when considering the upward movement from Q1 to Q2, even though 

In [15]:
class TeacherQuestion(Signature):
    prompt = InputField()
    question = OutputField(desc="A Socratic question to improve the prompt")

class TeacherQuestioner(Module):
    def __init__(self):
        super().__init__()
        self.generate = dspy.Predict(TeacherQuestion)

    def forward(self, prompt):
        return self.generate(prompt=prompt)


class CritiqueQuestion(Signature):
    question = InputField()
    critique = OutputField(desc="Is the question Socratic? Why or why not?")

class CriticJudge(Module):
    def __init__(self):
        super().__init__()
        self.evaluate = dspy.Predict(CritiqueQuestion)

    def forward(self, question):
        return self.evaluate(question=question)


class MarginAnalyzer(Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(AnalyzeMargins)

    def forward(self, context, question, teacher_question=None):
        if teacher_question:
            question = f"{question} Consider also: {teacher_question}"
        return self.predict(context=context, question=question)





In [17]:
# db.py
from sqlalchemy import create_engine, Column, Integer, String, Text
from sqlalchemy.orm import declarative_base, sessionmaker
import os

PG_CONN_STR = os.getenv("PG_CONN_STR")  # or hard-code for local dev

Base = declarative_base()
engine = create_engine(PG_CONN_STR)
SessionLocal = sessionmaker(bind=engine)

class MarsStep(Base):
    __tablename__ = "mars_steps"

    id = Column(Integer, primary_key=True, index=True)
    input_question = Column(Text)
    teacher_question = Column(Text)
    critique = Column(Text)
    final_question = Column(Text)
    final_answer = Column(Text)

def init_db():
    Base.metadata.create_all(bind=engine)


In [19]:

def log_mars_step(input_q, teacher_q, critique, final_q, final_a):
    session = SessionLocal()
    step = MarsStep(
        input_question=input_q,
        teacher_question=teacher_q,
        critique=critique,
        final_question=final_q,
        final_answer=final_a
    )
    session.add(step)
    session.commit()
    session.close()


In [20]:

init_db()  # only once at startup

input_q = "Is Tesla's profitability improving?"
t_out = teacher(prompt=input_q)
c_out = critic(question=t_out.question)

final_q = input_q if "no" in c_out.critique.lower() else f"{input_q} Consider: {t_out.question}"
s_out = student(context=context, question=input_q, teacher_question=t_out.question)

log_mars_step(
    input_q=input_q,
    teacher_q=t_out.question,
    critique=c_out.critique,
    final_q=final_q,
    final_a=s_out.answer
)


00:01:35 - LiteLLM:DEBUG: utils.py:308 - 

00:01:35 - LiteLLM:DEBUG: utils.py:308 - Request to litellm:
00:01:35 - LiteLLM:DEBUG: utils.py:308 - litellm.completion(cache={'no-cache': False, 'no-store': False}, retry_policy=RetryPolicy(BadRequestErrorRetries=0, AuthenticationErrorRetries=0, TimeoutErrorRetries=8, RateLimitErrorRetries=8, ContentPolicyViolationErrorRetries=8, InternalServerErrorRetries=8), max_retries=0, model='ollama_chat/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', messages=[{'role': 'system', 'content': 'Your input fields are:\n1. `prompt` (str)\n\nYour output fields are:\n1. `question` (str): A Socratic question to improve the prompt\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## prompt ## ]]\n{prompt}\n\n[[ ## question ## ]]\n{question}\n\n[[ ## completed ## ]]\n\nIn adhering to this structure, your objective is: \n        Given the fields `prompt`, produce the fields `question`.'}, {'role': 'user', 'content': "[

In [ ]:
dspy.inspect_history()





[2025-03-25T00:02:12.282935]

System message:

Your input fields are:
1. `context` (str): Relevant financial data
2. `question` (str): User's trading question

Your output fields are:
1. `answer` (str): Insightful, accurate answer

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `answer`.


User message:

[[ ## context ## ]]
Tesla Income Statements over 3 quarters...

[[ ## question ## ]]
Is Tesla's profitability improving? Consider also: To determine if Tesla's profitability is improving, what specific financial metrics should be analyzed, and how do these compare year-over-year?

Respond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker

: 

In [16]:
context = "Tesla Income Statements over 3 quarters..."
initial_question = "Is Tesla improving profitability?"

teacher = TeacherQuestioner()
critic = CriticJudge()
student = MarginAnalyzer()

# Teacher generates Socratic question
t_output = teacher(prompt=initial_question)
c_output = critic(question=t_output.question)

# Only use question if critic approves
if "yes" in c_output.critique.lower():
    answer = student(context=context, question=initial_question, teacher_question=t_output.question)
else:
    answer = student(context=context, question=initial_question)

print("🧠 Final Answer:", answer.answer)


23:51:38 - LiteLLM:DEBUG: utils.py:308 - 

23:51:38 - LiteLLM:DEBUG: utils.py:308 - Request to litellm:
23:51:38 - LiteLLM:DEBUG: utils.py:308 - litellm.completion(cache={'no-cache': False, 'no-store': False}, retry_policy=RetryPolicy(BadRequestErrorRetries=0, AuthenticationErrorRetries=0, TimeoutErrorRetries=8, RateLimitErrorRetries=8, ContentPolicyViolationErrorRetries=8, InternalServerErrorRetries=8), max_retries=0, model='ollama_chat/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', messages=[{'role': 'system', 'content': 'Your input fields are:\n1. `prompt` (str)\n\nYour output fields are:\n1. `question` (str): A Socratic question to improve the prompt\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## prompt ## ]]\n{prompt}\n\n[[ ## question ## ]]\n{question}\n\n[[ ## completed ## ]]\n\nIn adhering to this structure, your objective is: \n        Given the fields `prompt`, produce the fields `question`.'}, {'role': 'user', 'content': '[

🧠 Final Answer: To determine if Tesla is improving profitability, we need to analyze their income statements over the three quarters provided. Profitability can be assessed by looking at key financial metrics such as net income, gross profit margin, operating expenses, and revenue growth.

From the context:  
- **Q1 2023**: Net Income = $X million, Revenue = $Y billion  
- **Q2 2023**: Net Income = $Z million, Revenue = $A billion  
- **Q3 2023**: Net Income = $B million, Revenue = $C billion  

**Key Observations**:  
1. **Net Income Trend**: Compare net income across quarters to see if there's an upward or downward trend. For example, if Q3 net income is higher than Q2 and Q1, that indicates improvement.  
2. **Gross Profit Margin**: Calculate gross profit (Revenue - Cost of Goods Sold) for each quarter and compute the margin (gross profit / revenue). An increasing margin suggests cost efficiency improvements.  
3. **Operating Expenses**: Analyze operating expenses relative to revenu